# VectorStoreRetrieverMemory


`VectorStoreRetrieverMemory` 는 벡터 스토어에 메모리를 저장하고 호출될 때마다 가장 '눈에 띄는' 상위 K개의 문서를 쿼리합니다.

이는 대화내용의 순서를 **명시적으로 추적하지 않는다는 점** 에서 다른 대부분의 메모리 클래스와 다릅니다.


In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

먼저, 벡터 스토어를 초기화 합니다.


In [6]:
# import faiss
# from langchain_openai import OpenAIEmbeddings
# from langchain.docstore import InMemoryDocstore
# from langchain.vectorstores import FAISS


# # 임베딩 모델을 정의합니다.
# embeddings_model = OpenAIEmbeddings()

# # Vector Store 를 초기화 합니다.
# embedding_size = 1536
# index = faiss.IndexFlatL2(embedding_size)
# vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, AIMessage

# 1. faiss 설치 문제
# faiss는 window용 공식 wheel제공하지 않음
# pip 대신 prebuilt로 설치 가능, but python 버전이 3.11 미만이어야함
# langchain은 벡터 스토어가 다양하기 때문에 faiss 대신 Chroma로 대체 

embeddings = OpenAIEmbeddings()
vectorstore = Chroma(
    collection_name="chat_memory", 
    embedding_function=embeddings,
    persist_directory=None,
      
)




실제 사용에서는 `k`를 더 높은 값으로 설정하지만, 여기서는 `k=1` 을 사용하여 다음과 같이 표시합니다.


In [2]:

# # 벡터 조회가 여전히 의미적으로 관련성 있는 정보를 반환한다는 것을 보여주기 위해서입니다.
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
# memory = VectorStoreRetrieverMemory(retriever=retriever)



# 2. Retriever로 변환 (vectorStoreRetrieverMemory에서 사용)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


def save_dialogue(human, ai):
    vectorstore.add_texts([
        f"Human: {human}",
        f"AI: {ai}"
    ])



# 3. LLM + 프롬프트
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant. Use the memory below if helpful.\nMemory:\n{memory}"),
    ("human", "{question}")
])


# 4. Memory Retrieval → Prompt Binding → 모델 실행 체인

def retrieve_memory(question):
    docs = retriever.invoke(question)
    return "\n".join(d.page_content for d in docs)


chain = (
    {
        "memory": lambda x: retrieve_memory(x["question"]),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)



save_dialogue(
    "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.",
    "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다..."
)

save_dialogue(
    "프로젝트에서 어떤 역할을 맡았나요?",
    "제가 맡은 역할은 백엔드 개발자였습니다..."
)

save_dialogue(
    "팀 프로젝트에서 어려움을 겪었던 경험이 있다면?",
    "초기에 의사소통 문제로 어려움이 있었습니다..."
)

save_dialogue(
    "개발자로서 강점이 무엇인가요?",
    "빠른 학습 능력과 문제 해결 능력입니다..."
)





NameError: name 'vectorstore' is not defined

다음의 질문을 했을 때 Vector Store 로 부터 1개(k=1 이기 때문)의 가장 관련성 높은 대화를 반환합니다.

- 질문: "면접자 전공은 무엇인가요?"


In [1]:
# 메모리에 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.
# print(memory.load_memory_variables({"human": "면접자 전공은 무엇인가요?"})["history"])

query = "면접자 전공은 무엇인가요?"
response = chain.invoke({"question": query})

print(response.content)

NameError: name 'chain' is not defined

이번에는 다른 질문을 통해 가장 연관성 높은 1개 대화를 추출합니다.

- 질문: "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"


In [ ]:
# print(
#     memory.load_memory_variables(
#         {"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"}
#     )["history"]
# )

docs = retriever.invoke("면접자가 프로젝트에서 맡은 역할은 무엇인가요?")
history = "\n".join(d.page_content for d in docs)
print(history)
